This is an Image Classification Analysis, in which I am using images of cars and trucks to train my data and later make a prediction whether the given image is of a car or a truck.

The data I have used have seperate folders for Train and Test. Images of cars and trucks are further stored in seperate folders.
I am using Tensorflow library for creating neural network.



In [1]:
!pip install tensorflow
!pip install tflearn

My data is present on google drive and accessing the data using PyDrive.

In [2]:
!pip install PyDrive

    100% |████████████████████████████████| 993kB 23.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import cv2
import numpy as np
from random import shuffle

# providing size for resizing of images
img_size = 50

# Learning rate 
lr = 1e-3

# Giving Model Name having 6 Convonutional layers
model_name = 'cars vs truck-{}-{}.model'.format(lr,'6conv-basic')


Loading the train data of cars and trucks images

In [0]:
download = drive.CreateFile({'id': '1DeI5xZYA52WO2k5IiwHFSrMez5g2Tlj7'})

In [0]:
file_list = drive.ListFile({'q': "'1DeI5xZYA52WO2k5IiwHFSrMez5g2Tlj7' in parents"}).GetList()

In [0]:
# fetching every image from the folder and converting it to grayscale

# for every image of truck, introducing label [0,1] and [1,0] for car image.

# loading Truck train data

train = []
for f in file_list:
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(f['title'])
  img = cv2.resize(cv2.imread(f['title'],cv2.IMREAD_GRAYSCALE),(img_size,img_size))
  train.append([np.array(img),np.array([0,1])])

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1i9dJA5z-qhTDjHhCSMibSUMfgsC_uiXQ'})

In [0]:
file_list = drive.ListFile({'q': "'1i9dJA5z-qhTDjHhCSMibSUMfgsC_uiXQ' in parents"}).GetList()

In [0]:
# Car's train Data

for f in file_list:
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(f['title'])
  img = cv2.resize(cv2.imread(f['title'],cv2.IMREAD_GRAYSCALE),(img_size,img_size))
  train.append([np.array(img),np.array([1,0])])

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1ItW67KSt44Vsk55JUf4q0MFXwIY5UYSU'})

In [0]:
file_list = drive.ListFile({'q': "'1ItW67KSt44Vsk55JUf4q0MFXwIY5UYSU' in parents"}).GetList()

In [0]:
# Loading Test data in similar manner

test = []
for f in file_list:
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(f['title'])
  img = cv2.resize(cv2.imread(f['title'],cv2.IMREAD_GRAYSCALE),(img_size,img_size))
  test.append([np.array(img),np.array([1,0])])

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1zJpTtQyPmuRSkOqzRPMzvGSzbNzhDkwy'})

In [0]:
file_list = drive.ListFile({'q': "'1zJpTtQyPmuRSkOqzRPMzvGSzbNzhDkwy' in parents"}).GetList()

In [0]:
for f in file_list:
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(f['title'])
  img = cv2.resize(cv2.imread(f['title'],cv2.IMREAD_GRAYSCALE),(img_size,img_size))
  test.append([np.array(img),np.array([0,1])])

In [0]:
# Shuffeling Data for random occurance of cars and trucks

shuffle(train)
shuffle(test)

In [0]:
# For clearing the graph

from tensorflow import reset_default_graph
reset_default_graph()

Creating tflearn model with 6 layers. The more layers we introduce, we lower the value of Loss, but it should be causing any overfitting.So we should have balance between these.

In [24]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None,img_size,img_size,1],name='input')

convnet = conv_2d(convnet,32,2,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet,64,2,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet,32,2,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet,64,2,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet,32,2,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet,64,2,activation='relu')
convnet = max_pool_2d(convnet,2)

convnet = fully_connected(convnet,1024,activation='relu')
convnet = dropout(convnet,0.8)

convnet = fully_connected(convnet,2,activation='softmax')
convnet = regression(convnet,optimizer = 'adam', learning_rate = lr, loss = 'categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir = "log")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


Splitting data and labels 

In [0]:
X = np.array([i[0] for i in train]).reshape(-1,img_size,img_size,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,img_size,img_size,1)
test_y = [i[1] for i in test]

Time to fit our data in model we created earlier. I have provided epoch value to 5. And we can see the change in the Loss value and Accuracy with each epoch.

In [27]:
model.fit({'input':X},{'targets':Y},n_epoch=5,validation_set=({'input':test_x},{'targets':test_y}),snapshot_step = 500,show_metric=True,run_id=model_name)

---------------------------------
Run id: cars vs truck-0.001-6conv-basic.model
Log directory: log/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 20
Validation samples: 10
--
Training Step: 1  | time: 1.662s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 | val_loss: 11.51293 - val_acc: 0.5000 -- iter: 20/20
--
Training Step: 2  | total loss: 0.90697 | time: 1.080s
| Adam | epoch: 002 | loss: 0.90697 - acc: 0.4500 | val_loss: 9.12740 - val_acc: 0.5000 -- iter: 20/20
--
Training Step: 3  | total loss: 9.28938 | time: 1.073s
| Adam | epoch: 003 | loss: 9.28938 - acc: 0.4909 | val_loss: 1.91942 - val_acc: 0.5000 -- iter: 20/20
--
Training Step: 4  | total loss: 7.98277 | time: 1.074s
| Adam | epoch: 004 | loss: 7.98277 - acc: 0.4977 | val_loss: 2.28941 - val_acc: 0.5000 -- iter: 20/20
--
Training Step: 5  | total loss: 3.52730 | time: 1.076s
| Adam | epoch: 005 | loss: 3.52730 - acc: 0.4993 | v

In [28]:
model.save(model_name)

INFO:tensorflow:/content/cars vs truck-0.001-6conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.
